In [1]:
# ------------ Import Libraries ------------
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import csv
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
print("OK")

OK


In [2]:
# --------- Initialize Parameters -------------
num_classes = 10# dataset includes 10 classes
epochs = 20
learning_rate = 0.001
batch_size = 64
weight_decay = 0.0005
model_name = "cnn"
results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

# --------- Select CPU or GPU to run ----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

#--------- Check the name of the GPU --------
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")

Using device: cuda
GPU name: Tesla V100-SXM2-32GB


In [3]:
#---------- Processing Data --------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dir = "/home/users/dtran/sentry/eurosat/dataset/train"
val_dir = "/home/users/dtran/sentry/eurosat/dataset/test"

train_dataset = ImageFolder(root=train_dir, transform=transform)
val_dataset = ImageFolder(root=val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
print("OK")

OK


In [6]:
# ----------------- Model -----------------
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 256 * 28 * 28)  # Flatten the tensor
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Initialize the CNN model
model = CNN(num_classes=10)  # You can use a different number of classes based on your dataset
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

print("Model OK")

Model OK


In [7]:
# ----------------- Training -----------------
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{epochs}]")
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        loop.set_postfix(loss=loss.item(), accuracy=100 * correct / total)

    scheduler.step()
    
    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # Validation
    model.eval()
    val_loss = 0.0
    y_true, y_pred = [], []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    val_loss = val_loss / len(val_loader)
    val_losses.append(val_loss)

    val_accuracy = accuracy_score(y_true, y_pred) * 100
    val_accuracies.append(val_accuracy)

    print(f"\nEpoch {epoch+1} - Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%")
    print(f"\nEpoch {epoch+1} - Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%\n")

Epoch [1/20]: 100%|█████████████████████████████████████████████████| 338/338 [01:02<00:00,  5.43it/s, accuracy=54, loss=0.87]



Epoch 1 - Train Loss: 1.3019, Train Acc: 54.03%

Epoch 1 - Val Loss: 0.9495, Val Acc: 66.81%



Epoch [2/20]: 100%|██████████████████████████████████████████████| 338/338 [01:01<00:00,  5.53it/s, accuracy=71.8, loss=0.836]



Epoch 2 - Train Loss: 0.8016, Train Acc: 71.83%

Epoch 2 - Val Loss: 0.6662, Val Acc: 77.35%



Epoch [3/20]: 100%|██████████████████████████████████████████████| 338/338 [00:58<00:00,  5.76it/s, accuracy=77.7, loss=0.547]


...


Epoch 18 - Train Loss: 0.0570, Train Acc: 98.63%

Epoch 18 - Val Loss: 0.3387, Val Acc: 89.46%



Epoch [19/20]: 100%|█████████████████████████████████████████████| 338/338 [00:57<00:00,  5.91it/s, accuracy=98.8, loss=0.137]



Epoch 19 - Train Loss: 0.0539, Train Acc: 98.77%

Epoch 19 - Val Loss: 0.3395, Val Acc: 89.56%



Epoch [20/20]: 100%|████████████████████████████████████████████| 338/338 [00:57<00:00,  5.87it/s, accuracy=98.7, loss=0.0432]



Epoch 20 - Train Loss: 0.0554, Train Acc: 98.68%

Epoch 20 - Val Loss: 0.3433, Val Acc: 89.54%



In [8]:
#---------------- Save Model -----------------
torch.save(model.state_dict(), os.path.join(results_dir, f"{model_name}_trained.pth"))
print("Saved model")

Saved model


In [ ]:
# ----------------- Evaluation -----------------
start_time = time.time()
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

inference_time = time.time() - start_time

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
recall = recall_score(y_true, y_pred, average='macro', zero_division=0)

print(f"Final Results: Accuracy={accuracy:.4f}, Precision={precision:.4f}, Recall={recall:.4f}, Inference Time={inference_time:.2f}s")

In [ ]:
#--------------- Save Results to a csv file ------------------
csv_file = os.path.join(results_dir, "eval_results.csv")
file_exists = os.path.isfile(csv_file)

with open(csv_file, mode="a", newline="") as f:
    writer = csv.writer(f)
    if not file_exists:
        writer.writerow(["Model", "Accuracy", "Precision", "Recall", "Inference_Time"])
    writer.writerow([model_name, accuracy, precision, recall, inference_time])

print("Saved Results")

In [ ]:
# ----------------- Plotting -----------------
import numpy as np
# Train/Val Loss
plt.figure(figsize=(12,8), num=1)
plt.plot(range(1, epochs+1), train_losses, label='Train Loss')
plt.plot(range(1, epochs+1), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.xticks(np.arange(0, epochs+1, 5))
plt.legend()
plt.grid()
plt.title('Loss Curve')
plt.savefig(os.path.join(results_dir, "loss_curve_cnn.png"))

# Train/Val Accuracy
plt.figure()
plt.plot(range(1, epochs+1), train_accuracies, label='Train Accuracy')
plt.plot(range(1, epochs+1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.xticks(np.arange(0, epochs+1, 5))
plt.legend()
plt.grid()
plt.title('Accuracy Curve')
plt.savefig(os.path.join(results_dir, "accuracy_curve_cnn.png"))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10,8))
ConfusionMatrixDisplay(cm, display_labels=val_dataset.classes).plot(values_format='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xticks(rotation=90)
plt.savefig(os.path.join(results_dir, "confusion_matrix_cnn.png"))